In [ ]:
import numpy as np
import time
import os
import tensorflow as tf
from keras import optimizers
from keras.models import Sequential
from keras.constraints import non_neg
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, UpSampling1D, UpSampling2D, UpSampling3D
from keras.layers import Conv2D, Conv1D, MaxPooling2D, BatchNormalization
from keras.callbacks import TensorBoard
from keras.utils import np_utils
from keras.datasets import cifar10
from keras import backend
from scipy import misc
from keras.models import model_from_json
from skimage.transform import resize

##Modify Parameters Here
bands=4
hr_img_size=(450,450)
inpt_lr_img_size=(135,135)
sptlvarmeasure=32

In [ ]:
def array_from_tif_dir(tifdir,res):
    finaldim=(len(os.listdir(os.getcwd() + '/tifs/' + tifdir)),res,res,4)
    finalset=np.zeros(finaldim)
    imgcounter=0
    for imagename in os.listdir(os.getcwd() + '/tifs/' + tifdir):
        image = misc.imread(os.getcwd() + '/tifs/' + tifdir + imagename)
        finalset[imgcounter,:,:,:]= image
        imgcounter+=1
    return(finalset)
y_train=array_from_tif_dir('HR/',hr_img_size[0])
y_test=array_from_tif_dir('HRV/',hr_img_size[0])
X_train=array_from_tif_dir('LR/',inpt_lr_img_size[0])
X_test=array_from_tif_dir('LRV/',inpt_lr_img_size[0])
        
        

In [ ]:


# 4. Load pre-shuffled MNIST data into train and test sets
# newximglist=[]
# newx2imglist=[]
# imagecounter=0
# # for image in X_train:
# #     preimage=np.swapaxes(image.copy(),0,-1)
# #     newximage=misc.imresize(preimage.copy(), inpt_lr_img_size)
# #     finalimage=np.swapaxes(newximage.copy(),0,-1)
# #     newximglist.append(finalimage.copy())
# #     imagecounter+=1
# # X_train=np.array(newximglist)
# # for image in X_test:
# #     preimage=np.swapaxes(image.copy(),0,-1)
# #     newximage=misc.imresize(preimage.copy(), inpt_lr_img_size)
# #     finalimage=np.swapaxes(newximage.copy(),0,-1)
# #     newx2imglist.append(finalimage.copy())
# #     imagecounter+=1
# X_test=np.array(newx2imglist)
y_train=np.swapaxes(y_train,1,-1)
y_test=np.swapaxes(y_test,1,-1)
X_train=np.swapaxes(X_train,1,-1)
X_test=np.swapaxes(X_test,1,-1)
print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)
# 5. Preprocess input data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')[0:10] 
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')[0:10]
X_train /= 255
X_test /= 255
y_train /= 255
y_test /= 255


In [ ]:

 
# 7. Define model architecture
model = Sequential()
ilis=inpt_lr_img_size[0]
ihis=hr_img_size[0]
ratio=float(ilis)/float(ihis)
collapsesize=int((ratio**3)*ihis)
solved=int((ilis-collapsesize+10)/8)

model.add(Conv2D(filters=bands*3, kernel_size=(solved, solved), data_format='channels_first', activation='relu', input_shape=(bands,ilis,ilis)))
model.add(Conv2D(filters=bands*3, kernel_size=(solved, solved), data_format='channels_first', activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.1))

model.add(Conv2D(filters=bands*5, kernel_size=(solved, solved), data_format='channels_first', activation='relu'))
model.add(Conv2D(filters=bands*5, kernel_size=(solved, solved), data_format='channels_first', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters=bands*7, kernel_size=(solved, solved), data_format='channels_first', activation='relu'))
model.add(Conv2D(filters=bands*7, kernel_size=(solved, solved), data_format='channels_first', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(filters=bands*12, kernel_size=(solved/2, solved/2), data_format='channels_first', activation='relu'))
model.add(Conv2D(filters=bands*12, kernel_size=(solved/2, solved/2), data_format='channels_first', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(filters=bands*15, kernel_size=(solved/2, solved/2), data_format='channels_first', activation='relu'))
model.add(Conv2D(filters=bands*15, kernel_size=(solved/2, solved/2), data_format='channels_first', activation='relu'))
currentsize=model.layers[-1].get_output_at(0).get_shape().as_list()[-1]
reduceneed=currentsize-collapsesize
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(filters=bands*15, kernel_size=(reduceneed+1, reduceneed+1), data_format='channels_first', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(UpSampling2D((int(1/ratio*3),int(1/ratio*3)),data_format='channels_first'))
print(int(1/ratio*3))
model.add(Conv2D(filters=bands*15, kernel_size=(solved/2, solved/2), data_format='channels_first', kernel_constraint=non_neg(), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=bands*15, kernel_size=(solved/2, solved/2), data_format='channels_first', kernel_constraint=non_neg(), activation='relu'))
model.add(BatchNormalization())


model.add(UpSampling2D((int(1/ratio),int(1/ratio)),data_format='channels_first'))
print(int(1/ratio*2))
model.add(Conv2D(filters=bands*12, kernel_size=(solved/2, solved/2), data_format='channels_first', kernel_constraint=non_neg(), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=bands*12, kernel_size=(solved/2, solved/2), data_format='channels_first', kernel_constraint=non_neg(), activation='relu'))
model.add(BatchNormalization())

model.add(UpSampling2D((2,2),data_format='channels_first'))
model.add(Conv2D(filters=bands*5, kernel_size=(solved/2, solved/2), data_format='channels_first', kernel_constraint=non_neg(), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=bands*5, kernel_size=(solved/2, solved/2), data_format='channels_first', kernel_constraint=non_neg(), activation='softmax'))
currentsize=model.layers[-1].get_output_at(0).get_shape().as_list()[-1]
bandcollapse=model.layers[-1].get_output_at(0).get_shape().as_list()[1]
reduceneed=currentsize-hr_img_size[0]
model.add(BatchNormalization())

model.add(Conv2D(filters=bands, kernel_size=(reduceneed+1, reduceneed+1), data_format='channels_first', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))




 
# 8. Compile model
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])





In [ ]:
print(model.summary())

In [ ]:
if os.path.isfile('model.json'):
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
opt = optimizers.Adam(lr=.05, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
print(X_train.shape)
tensorboard = TensorBoard(log_dir=os.getcwd()+'/logs'.format(time.time()))
writer = tf.summary.FileWriter(os.getcwd()+'/logs').add_graph(sess.graph)
try:
  loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
  loaded_model.fit(X_train, y_train, 
              batch_size=1, nb_epoch=1, verbose=1, callbacks=[tensorboard])
  model=loaded_model
except (NameError, ValueError) as e:
    model.fit(X_train, y_train, 
              batch_size=1, nb_epoch=1, verbose=1, callbacks=[tensorboard])
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")




In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
class_scores= model.predict(X_test)
print(score)

In [ ]:
import matplotlib.pyplot as plt
classes=[]
for image in class_scores:
    thisclass=image
    classes.append(thisclass)
f, ax = plt.subplots(10,3, figsize=(20, 16))
axind0=0
axind1=0
imagecounter=0
for image in X_test:
    axind0=imagecounter
    emptyximage=np.zeros((10,10,3))
    emptyyimage=np.zeros((32,32,3))
    emptytargetimage=np.zeros((32,32,3))
    for band in range(1,4):
        newximage=np.abs(image[band-1:band,0:10,0:10])
        newtargetimage=y_test[imagecounter][band-1:band,0:32,0:32]
        newyimage=np.abs(classes[imagecounter][band-1:band,0:32,0:32])
        emptyximage[:,:,band-1:band]=newximage.reshape((10,10,1))*(1/np.max(newximage))
        emptyyimage[:,:,band-1:band]=newyimage.reshape((32,32,1))*(1/np.max(newyimage))
        emptytargetimage[:,:,band-1:band]=newtargetimage.reshape((32,32,1))*(1/np.max(newtargetimage))
    imagecounter+=1
    ax[axind0,0].imshow(emptyximage)
    ax[axind0,1].imshow(emptyyimage)
    ax[axind0,2].imshow(emptytargetimage)
    
    
    